In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# GPU 사용가능 여부 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2024-04-16 09:46:52.364885: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2024-04-16 09:46:52.386480: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799925000 Hz
2024-04-16 09:46:52.387641: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x45ff900 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-16 09:46:52.387652: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13067585264966636066,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7381625496735809623
 physical_device_desc: "device: XLA_CPU device"]

In [3]:
#파이썬 버전 확인
import sys
print(sys.version)

3.7.16 (default, Jan 17 2023, 22:20:44) 
[GCC 11.2.0]


In [4]:
#텐서플로우 버전 확인
import tensorflow as tf
tf.__version__

'1.15.4'

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "/device:GPU:0"

In [6]:
# common library
import pandas as pd
import numpy as np
import time
from stable_baselines.common.vec_env import DummyVecEnv
# config
from config.config_tse import *
# preprocessor
from preprocessing.preprocessors_tse import * # from folder.file import *
# model
from model.models_tse import *

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



No protocol specified


In [7]:
def run_model() -> None:
    """Train the model."""

    # read and preprocess data

    preprocessed_path = "data/tse30_done_data.csv"
    if os.path.exists(preprocessed_path):
        data = pd.read_csv(preprocessed_path, index_col=0) # 0번째 column을 인덱스로 지정
    else:
        data = preprocess_data()
        data = add_turbulence(data)
        data.to_csv(preprocessed_path)

    # print(data.head())
    # print(data.size)

    # 2015/10/01 is the date that validation starts
    # 2016/01/01 is the date that real trading starts
    # unique_trade_date needs to start from 2015/10/01 for validation purpose
    
    unique_trade_date = data[(data.datadate > 20151001)].datadate.unique()
    #print(unique_trade_date)

    # rebalance_window is the number of months to retrain the model
    # validation_window is the number of months to validation the model and select for trading
    rebalance_window = 63 # default=63
    validation_window = 63 # default=63
    
    
    ## Original Ensemble Strategy
    #run_ensemble_strategy(df=data, 
    #                      unique_trade_date= unique_trade_date,
    #                      rebalance_window = rebalance_window,
    #                      validation_window= validation_window)
    
    ## Remake Ensemble
    #run_remake_ensemble(df=data, 
    #              unique_trade_date= unique_trade_date,
    #              rebalance_window = rebalance_window,
    #              validation_window= validation_window)
    
    ## Renewal Ensemble2
    #run_ensemble2(df=data, 
    #              unique_trade_date= unique_trade_date,
    #              rebalance_window = rebalance_window,
    #              validation_window= validation_window)
    

    ## PPO
    #run_ppo(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## A2C
    run_a2c(df=data, 
            unique_trade_date= unique_trade_date,
            rebalance_window = rebalance_window,
            validation_window= validation_window)
    
    ## DDPG
    #run_ddpg(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #       validation_window= validation_window)
    
    ## ACKTR
    #run_acktr(df=data, 
    #       unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #       validation_window= validation_window)
    
    ## TRPO
    #run_trpo(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## SAC
    #run_sac(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## TD3
    #run_td3(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)
    
    ## GAIL
    #run_gail(df=data, 
    #        unique_trade_date= unique_trade_date,
    #        rebalance_window = rebalance_window,
    #        validation_window= validation_window)

In [ ]:
#_logger.info(f"saving model version: {_version}")
if __name__ == "__main__":
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        run_model()